## Importing all libraries


In [1]:
import os
import pathlib
import numpy as np
import string
import glob
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import Counter
from torch.utils.data import DataLoader, Dataset, Subset

plt.style.use("ggplot")

/home/veer/Documents/model-hub/simple_text_classifier/venv/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting seeds for reproducibility


In [2]:
def set_seed(seed=1234):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # for multiple gpus
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


set_seed()

In [3]:
OUTPUT_DIR = os.path.join("outputs", "imdb_movie_review_embedding")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [4]:
data_dir = os.path.join("input", "aclImdb")
dataset_dir = os.path.join(data_dir, "aclImdb")
train_dir = os.path.join(dataset_dir, "train")
print(f"The dataset directory is {dataset_dir}")
print(f"The train directory is {train_dir}")

The dataset directory is input/aclImdb/aclImdb
The train directory is input/aclImdb/aclImdb/train


### Preprocessing


In [17]:
def find_longest_length(text_file_paths):
    """
    Find the longest review length in the entire training set.
    """
    longest = 0
    for path in text_file_paths:
        with open(path, "r") as f:
            text = f.read()
            # Remove <br> tags.
            text = re.sub("<[^>]+>+", "", text)
            corpus = [word for word in text.split()]
        if len(corpus) > longest:
            longest = len(corpus)
    return longest

In [18]:
file_paths = []
file_paths.extend(glob.glob(os.path.join(dataset_dir, "train", "pos", "*.txt")))
file_paths.extend(glob.glob(os.path.join(dataset_dir, "train", "neg", "*.txt")))
longest_sentence_length = find_longest_length(file_paths)
print(f"Longest review length: {longest_sentence_length} words")

Longest review length: 2450 words


In [42]:
def find_avg_length(text_file_paths):
    """
    Find the average review size in the training
    """
    all_lens = []
    for path in text_file_paths:
        with open(path, "r") as f:
            text = f.read()
            # Remove <br> tag
            text = re.sub("<[^>]+>+", "", text)
            corpus = [word for word in text.split()]
        all_lens.append(len(corpus))
    np_array = np.array(all_lens)
    return np.mean(np_array)

In [44]:
file_paths = []
file_paths.extend(glob.glob(os.path.join(dataset_dir, "train", "pos", "*.txt")))
file_paths.extend(glob.glob(os.path.join(data_dir, "train", "neg", "*.txt")))
average_length = find_avg_length(file_paths)
print(f"Average review length: {average_length} words")

Average review length: 232.76296 words


### Defining some general hyper parameters


In [72]:
MAX_LEN = int(longest_sentence_length)
NUM_WORDS = -1
BATCH_SIZE = 512
VALID_SPLIT = 0.25

In [73]:
def find_word_frequency(text_file_paths, most_common=None):
    """
    Returns a list of tuples, (<word>,<frequency>)
    """
    corpus = []
    for path in text_file_paths:
        with open(path, "r") as f:
            text = f.read()
            text = re.sub("<^>+>+", "", text)
            corpus.extend([word for word in text.split()])
    count_words = Counter(corpus)
    word_frequency = count_words.most_common(n=most_common)
    return word_frequency

In [74]:
 def word2int(input_words, num_words):
    """
        int_mapping: A dictionary of word and a integer mapping as 
            key-value pair. Example, {'Hello,': 1, 'the': 2, 'let': 3}
    """
    if num_words > -1:
        int_mapping = {
            w:i+1 for i, (w, c) in enumerate(input_words) \
                if i <= num_words - 1 # -1 to avoid getting (num_words + 1) integer mapping.
        }
    else:
        int_mapping = {w:i+1 for i, (w, c) in enumerate(input_words)}
    return int_mapping

### Creating a pytorch dataset


In [83]:
class ClassificationDataset(Dataset):
    def __init__(self, file_paths, word_frequency, int_mapping, max_len) -> None:
        self.word_frequency = word_frequency
        self.int_mapping = int_mapping
        self.max_len = max_len
        self.file_paths = file_paths

    def standardize_text(self, input_text: str) -> str:
        text = input_text.lower()
        text = re.sub(r"<^>+>+", "", text)
        text = "".join(
            [character for character in text if character not in string.punctuation]
        )
        return text

    def return_int_vector(self, int_mapping, text_file_path):
        """
        Map word to int in list
        """
        with open(text_file_path, "r") as f:
            text = f.read()
            text = self.standardize_text(text)
            corpus = [word for word in text.split()]
        int_vector = [int_mapping[word] for word in text.split() if word in int_mapping]
        return int_vector

    def pad_features(self, int_vector, max_len):
        """
        Return features of `int_vector`, where each vector is padded
        with 0's or truncated to the input seq_length. Return as Numpy
        array.
        """
        features = np.zeros((1, max_len), dtype=int)
        if len(int_vector) <= max_len:
            zeros = list(np.zeros(max_len - len(int_vector)))
            new = zeros + int_vector
        else:
            new = int_vector[:max_len]
        features = np.array(new)
        return features

    def encode_labels(self, text_file_path):
        file_path = pathlib.Path(text_file_path)
        class_label = str(file_path).split(os.path.sep)[-2]
        if class_label == "pos":
            int_label = 1
        else:
            int_label = 0
        return int_label

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        int_vector = self.return_int_vector(self.int_mapping, file_path)
        padded_features = self.pad_features(int_vector, self.max_len)
        label = self.encode_labels(file_path)
        return {
            "text": torch.tensor(padded_features, dtype=torch.int32),
            "label": torch.tensor(label, dtype=torch.long),
        }

In [84]:
file_paths = []
file_paths.extend(glob.glob(os.path.join(
    dataset_dir, 'train', 'pos', '*.txt'
)))
file_paths.extend(glob.glob(os.path.join(
    dataset_dir, 'train', 'neg', '*.txt'
)))
test_file_paths = []
test_file_paths.extend(glob.glob(os.path.join(
    dataset_dir, 'test', 'pos', '*.txt'
)))
test_file_paths.extend(glob.glob(os.path.join(
    dataset_dir, 'test', 'neg', '*.txt'
)))

In [85]:
word_frequency = find_word_frequency(file_paths)
int_mapping = word2int(word_frequency, num_words=NUM_WORDS)
print(f'The word freq is {word_frequency}')

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [86]:
dataset = ClassificationDataset(
    file_paths, word_frequency, int_mapping, MAX_LEN
)
dataset_size = len(dataset)
# Calculate the validation dataset size.
valid_size = int(VALID_SPLIT*dataset_size)
# Radomize the data indices.
indices = torch.randperm(len(dataset)).tolist()
# Training and validation sets.
dataset_train = Subset(dataset, indices[:-valid_size])
dataset_valid = Subset(dataset, indices[-valid_size:])
dataset_test = ClassificationDataset(
    test_file_paths, word_frequency, int_mapping, MAX_LEN
)
# dataset_valid = NLPClassificationDataset()
print(f"Number of training samples: {len(dataset_train)}")
print(f"Number of validation samples: {len(dataset_valid)}")
print(f"Number of test samples: {len(dataset_test)}")

Number of training samples: 18750
Number of validation samples: 6250
Number of test samples: 25000


In [87]:
int_to_word_train = {value: key for key, value in int_mapping.items()}
inputs = ''
for x in dataset_train[0]['text']:
    if x != 0:
        inputs += ' ' + int_to_word_train[int(x)]
print(inputs)
print('#'*25)
if int(dataset_train[0]['label']) == 1:
    label = 'Positive'
else:
    label = 'Negative'
print('Label:', label)

 what a stunning episode for this fine series this is television excellence at its best the story takes place in 1968 and its beautifully filmed in black white almost a film noir style with its deep shadows and stark images this is a story about two men who fall in love but i dont want to spoil this it is a rare presentation of what homosexuals faced in the 1960s in america written by the superb tom and directed by the great we move through their lives their love for each other and their tragedy taking on such a sensitive issue makes this episode all the more stunning our emotions are as torn and on edge as the characters chills ran up my spine at the end when they played bob gorgeous ah but i was so much older then im younger than that now as sung by the this one goes far past a 10 and all the way to the stars beautiful
#########################
Label: Positive


### Preparing the train and test data loader

In [88]:
train_loader = DataLoader(
    dataset_train, 
    batch_size=BATCH_SIZE,
    shuffle=True, 
    num_workers=4
)
valid_loader = DataLoader(
    dataset_valid, 
    batch_size=BATCH_SIZE,
    shuffle=False, 
    num_workers=4
)
test_loader = DataLoader(
    dataset_test, 
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

In [89]:
def binary_accuracy(labels, outputs, train_running_correct):
    outputs = torch.sigmoid(outputs)
    running_correct = 0
    for i, label in enumerate(labels):
        if label < 0.5 and outputs[i] < 0.5:
            running_correct += 1
        elif label >= 0.5 and outputs[i] >= 0.5:
            running_correct += 1
    return running_correct

In [90]:
def train(model, trainloader, optimizer, criterion, device):
    model.train()
    print('Training........')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        counter += 1
        inputs, labels = data['text'], data['label']
        inputs = inputs.to(device)
        labels = torch.tensor(labels, dtype=torch.float32).to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.squeeze(outputs, -1)
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        running_correct = binary_accuracy(
            labels, outputs, train_running_correct
        )
        train_running_correct += running_correct
        loss.backward()
        optimizer.step()
    
    epoch_loss = train_running_loss / counter
    epoch_acc = 100. * (train_running_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc
# Validation function.
def validate(model, testloader, criterion, device):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    valid_running_correct = 0
    counter = 0
    
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            inputs, labels = data['text'], data['label']
            inputs = inputs.to(device)
            labels = torch.tensor(labels, dtype=torch.float32).to(device)
            outputs = model(inputs)
            outputs = torch.squeeze(outputs, -1)
            loss = criterion(outputs, labels)
            valid_running_loss += loss.item()
            running_correct = binary_accuracy(
                labels, outputs, valid_running_correct
            )
            valid_running_correct += running_correct
        
    epoch_loss = valid_running_loss / counter
    epoch_acc = 100. * (valid_running_correct / len(testloader.dataset))
    return epoch_loss, epoch_acc

In [95]:
 class TextEmbedding(nn.Module):
    def __init__(self, vocab_size, max_len, embed_dim):
        super(TextEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=embed_dim)
        self.linear1 = nn.Linear(max_len, 1)
        
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        bs, _, _ = x.shape
        x = F.adaptive_avg_pool1d(x, 1).reshape(bs, -1)
        out = self.linear1(x)
        return out   

In [96]:
EMBED_DIM = 50
model = TextEmbedding(
    len(int_mapping)+1, 
    MAX_LEN,
    EMBED_DIM
).to(device)

### Actually Training the model

In [97]:
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(
    model.parameters(), 
    lr=0.001,
)
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

TextEmbedding(
  (embedding): Embedding(280618, 50)
  (linear1): Linear(in_features=2450, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
14,033,351 total parameters.
14,033,351 training parameters.



In [98]:
epochs = 50
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss, train_epoch_acc = train(model, train_loader, 
                                            optimizer, criterion, device)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_loader,  
                                                criterion, device)
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    print(f"Training loss: {train_epoch_loss}, training acc: {train_epoch_acc}")
    print(f"Validation loss: {valid_epoch_loss}, validation acc: {valid_epoch_acc}")
    # Save model.
    torch.save(
        model, os.path.join(OUTPUTS_DIR, 'model.pth')
    )
    print('-'*50)

Epoch 1 of 50
Training........


  0%|                                                                      | 0/37 [00:00<?, ?it/s]/tmp/ipykernel_65913/2891472870.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32).to(device)
100%|█████████████████████████████████████████████████████████████| 37/37 [00:04<00:00,  8.13it/s]

Validation



  0%|                                                                      | 0/13 [00:00<?, ?it/s]/tmp/ipykernel_65913/2891472870.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float32).to(device)
100%|█████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 13.52it/s]


Training loss: 0.6971754512271365, training acc: 50.34133333333334
Validation loss: 0.6925831849758441, validation acc: 51.632
--------------------------------------------------
Epoch 2 of 50
Training........


...

Training loss: 0.16295965237391963, training acc: 94.128
Validation loss: 0.2843192689693891, validation acc: 88.128
--------------------------------------------------
Epoch 50 of 50
Training........


100%|█████████████████████████████████████████████████████████████| 37/37 [00:04<00:00,  7.80it/s]

Validation



100%|█████████████████████████████████████████████████████████████| 13/13 [00:01<00:00, 11.37it/s]


Training loss: 0.15676193382288958, training acc: 94.17066666666668
Validation loss: 0.28474500660712904, validation acc: 88.144
--------------------------------------------------


In [ ]:
def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='blue', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='red', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(OUTPUTS_DIR, 'accuracy.png'))
    plt.show()
    # Loss plots.
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='blue', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(OUTPUTS_DIR, 'loss.png'))
    plt.show() 
save_plots(train_acc, valid_acc, train_loss, valid_loss)